In [ ]:
(* #use "topfind";; *)
(* #require "ppx_deriving";; *)
(* #require "ppx_jane";; *)
#require "yojson";;
#require "ppx_yojson_conv";;
(* #require "core";; *)
#require "async";;
#require "cohttp-async";;
#require "async_ssl";;
#require "ppx_let" ;;

#require "owl-top, jupyter.notebook";;
#require "jupyter-archimedes";;
#require "owl-jupyter";;
#require "owl-plplot";;

(* open Owl_jupyter;; *)
(* open Owl;; *)

(* #thread ;;
let () = Thread_safe.block_on_async_exn (fun () -> get_token "15011272962" "u123456A") *)

In [81]:


module JQData : sig
  val get_token : string -> string -> string
  val get_security_info : string -> string -> Owl.Dataframe.t
  val get_price : string -> string -> int -> string -> string -> Owl.Dataframe.t
end = struct
    
    let jqapi_uri = Uri.of_string "https://dataapi.joinquant.com/apis"
    
    let get_data req =
        let open Async in
        let open Cohttp in
        Thread_safe.block_on_async_exn (fun () ->
        let req_body = Cohttp_async.Body.of_string req in
        try_with(
            fun () -> 
            Cohttp_async.Client.post ~body:req_body jqapi_uri
            >>= fun (resp, body) ->
                Cohttp_async.Body.to_string body 
                >>| fun body -> body
        )
        >>| function
          | Ok (data)   -> data
          | Error _ -> ""
        );;
    
    let get_token username password =
      Format.sprintf {|{"method":"get_token","mob":"%s","pwd":"%s"}|} username password |> get_data;;
          
    let get_security_info token code =
        Format.sprintf {|{"method":"get_security_info","token":"%s","code":"%s"}|} token code 
        |> get_data |> DataframeUtils.of_csv_str;;
        
    let get_price token code count data_unit end_date = 
        let req = Format.sprintf
            {|{"method": "get_price","token": "%s","code": "%s","count": %d,"unit": "%s","end_date": "%s"}|}
            token code count data_unit end_date in
        get_data req |> DataframeUtils.of_csv_str;;
end


module JQData :
  sig
    val get_token : string -> string -> string
    val get_security_info : string -> string -> Owl.Dataframe.t
    val get_price :
      string -> string -> int -> string -> string -> Owl.Dataframe.t
  end


In [78]:

module DataframeUtils : sig
  val of_csv_str : ?sep:char -> ?head:string array -> ?types:string array -> string -> Owl.Dataframe.t
end = struct

let str_to_elt_fun = function
  | "b" -> fun a -> Owl.Dataframe.Bool (bool_of_string a)
  | "i" -> fun a -> Owl.Dataframe.Int (int_of_string a)
  | "f" -> fun a -> if a = "" then Owl.Dataframe.Float Float.nan else Owl.Dataframe.Float (float_of_string a)
  | "s" -> fun a -> Owl.Dataframe.String a
  | _   -> failwith "str_to_elt_fun: unsupported type";;

let guess_separator lines =
  let open Owl in
  let sep = [| ','; ' '; '\t'; ';'; ':'; '|' |] in
  (* rank by dividing as many parts as possible *)
  let tmp =
    Array.map
      (fun c ->
        let l = String.split_on_char c lines.(0) in
        c, List.length l)
      sep
  in
  (* sort by decreasing order *)
  Array.sort (fun a b -> snd b - snd a) tmp;
  let sep = Array.map fst tmp in
  let not_sep = ref true in
  let sep_idx = ref 0 in
  while !not_sep = true do
    let c = sep.(!sep_idx) in
    let n = String.split_on_char c lines.(0) |> List.length in
    (try
       Array.iter
         (fun line ->
           let m = String.split_on_char c line |> List.length in
           if m <> n then raise Owl_exception.FOUND)
         lines;
       not_sep := false
     with
    | _exn -> ());
    if !not_sep = true then sep_idx := !sep_idx + 1
  done;
  (* if cannot detect, return comma as default sep *)
  if !not_sep = false then sep.(!sep_idx) else ',';;


let guess_types sep lines =
  let open Owl in
  (* Note: no need to add "s" since it is default type *)
  let typ = [| "b"; "i"; "f" |] in
  let num_lines = Array.length lines in
  (* at least two lines because the first one will be dropped *)
  assert (num_lines > 1);
  let num_cols = lines.(0) |> String.trim |> String.split_on_char sep |> List.length in
  (* split into separate columns *)
  let stacks = Array.init num_cols (fun _ -> Owl_utils_stack.make ()) in
  Array.iteri
    (fun i line ->
      if i > 0
      then
        String.trim line
        |> String.split_on_char sep
        |> List.iteri (fun i c -> Owl_utils_stack.push stacks.(i) c))
    lines;
  let cols = Array.map Owl_utils_stack.to_array stacks in
  (* guess the types of columns *)
  Array.mapi
    (fun _i col ->
      let guess_typ = ref "s" in
      (try
         Array.iter
           (fun col_typ ->
             let typ_fun = str_to_elt_fun col_typ in
             let wrong_guess = ref false in
             (try
                Array.iter
                  (fun x ->
                    let y = String.trim x in
                    typ_fun y |> ignore)
                  col
              with
             | _exn -> wrong_guess := true);
             if !wrong_guess = false
             then (
               guess_typ := col_typ;
               raise Owl_exception.FOUND))
           typ
       with
      | _exn -> ());
      !guess_typ)
    cols;;

let csv_head ?(sep = '\t') head_i lines =
(*     head_i |> ignore;; *)
    let line = lines.(0) in
    line |> String.trim |> String.split_on_char sep |> Array.of_list;;

let of_csv_str ?sep ?head ?types str =
  let open Owl in
  let lines = Array.of_list @@ String.split_on_char '\n' str in
  let count = Array.length lines in
  let sep =
    match sep with
    | Some a -> a
    | None   -> guess_separator lines
  in
  let head_i = 0 in
  let head_names =
    match head with
    | Some a -> a
    | None   -> csv_head ~sep head_i lines
  in
  let types =
    match types with
    | Some a -> a
    | None   -> guess_types sep lines
  in
  assert (Array.length head_names = Array.length types);
  let convert_f = Array.map str_to_elt_fun types in
  let dataframe = Dataframe.make head_names in
  let dropped_line = ref 0 in
  for i = head_i + 1 to count - 1 do
    try
        let lines2 = lines.(i) |> String.split_on_char sep |> Array.of_list  in
        let row = Array.map2 (fun f a -> f a) convert_f lines2 in
        Dataframe.append_row dataframe row
    with
      | _exn ->
        dropped_line := !dropped_line + 1;
        Owl_log.warn "%s" @@ Printexc.to_string _exn;
        Owl_log.warn "of_csv_str: fail to parse line#%i" i
  done;
  
  if !dropped_line > 0 then Owl_log.warn "of_csv_str: %i lines have been dropped" !dropped_line;
  dataframe;;
end

module DataframeUtils :
  sig
    val of_csv_str :
      ?sep:char ->
      ?head:string array -> ?types:string array -> string -> Owl.Dataframe.t
  end


In [83]:
let token = JQData.get_token "15011272962" "u123456A";;
let info = JQData.get_security_info token "502050.XSHG";;
let df = JQData.get_price token "600000.XSHG" 20 "1d" "2021-11-08";;

let open Owl in
let open Owl_plplot in
let open Owl_jupyter in

let h = Plot.create "plot_024.png" in
(* Plot.(plot_fun ~h ~spec:[ RGB (0,0,255); Marker "#[0x2299]"; MarkerSize 2. ] Maths.sin 0. 9.); *)
Plot.(plot ~h ~spec:[ RGB (255,0,0); LineStyle 1; Marker "#[0x2299]"; MarkerSize 2. ] x y0);
Plot.output h;;
(* Owl_pretty.pp_dataframe Format.std_formatter df;;
flush stdout;; *)

val token : string = "5b6a9ba2b7f07fb129667f2f04cb0ab30171ae50"


val info : Owl.Dataframe.t =
  
  +-----------+------------+-----+----------+----------+----+-----------
          code display_name  name start_date   end_date type      parent 
  +-----------+------------+-----+----------+----------+----+-----------
R0 502050.XSHG    上证50B SZ50B 2015-04-27 2021-01-04  fjb 502048.XSHG 



val df : Owl.Dataframe.t =
  
   +----------+----+-----+----+----+---+------+----------+---------+----+---------
          date open close high  low     paused high_limit low_limit  avg pre_close 
   +----------+----+-----+----+----+---+------+----------+---------+----+---------
 R0 2021-10-12 9.12  9.14 9.19 9.09 ...      0      10.07      8.24 9.14      9.15 
 R1 2021-10-13 9.14  9.07 9.15 9.05 ...      0      10.05      8.23 9.09      9.14 
 R2 2021-10-14 9.07  9.04 9.09 9.02 ...      0       9.98      8.16 9.05      9.07 
 R3 2021-10-15 9.05  9.07 9.11 9.02 ...      0       9.94      8.14 9.07      9.04 
 R4 2021-10-18 9.06  9.02  9.1 8.98 ...      0       9.98      8.16 9.02      9.07 
 R5 2021-10-19 9.03  9.01 9.07 9.01 ...      0       9.92      8.12 9.03      9.02 
 R6 2021-10-20 9.03  9.02 9.06 8.99 ...      0       9.91      8.11 9.02      9.01 
 R7 2021-10-21 9.05  9.04 9.09 9.01 ...      0       9.92      8.12 9.05      9.02 
 R8 2021-10-22 9.07  9.07 9.13 9.02 ...      0  



   +----------+----+-----+----+----+---+------+----------+---------+----+---------
          date open close high  low     paused high_limit low_limit  avg pre_close 
   +----------+----+-----+----+----+---+------+----------+---------+----+---------
 R0 2021-10-12 9.12  9.14 9.19 9.09 ...      0      10.07      8.24 9.14      9.15 
 R1 2021-10-13 9.14  9.07 9.15 9.05 ...      0      10.05      8.23 9.09      9.14 
 R2 2021-10-14 9.07  9.04 9.09 9.02 ...      0       9.98      8.16 9.05      9.07 
 R3 2021-10-15 9.05  9.07 9.11 9.02 ...      0       9.94      8.14 9.07      9.04 
 R4 2021-10-18 9.06  9.02  9.1 8.98 ...      0       9.98      8.16 9.02      9.07 
 R5 2021-10-19 9.03  9.01 9.07 9.01 ...      0       9.92      8.12 9.03      9.02 
 R6 2021-10-20 9.03  9.02 9.06 8.99 ...      0       9.91      8.11 9.02      9.01 
 R7 2021-10-21 9.05  9.04 9.09 9.01 ...      0       9.92      8.12 9.05      9.02 
 R8 2021-10-22 9.07  9.07 9.13 9.02 ...      0       9.94      8.14 9.06    

In [85]:
let x = Owl.Mat.linspace 0. 2. 100

val x : Owl.Mat.mat =
  
   C0       C1       C2        C3        C4         C95     C96    C97    C98 C99 
R0  0 0.020202 0.040404 0.0606061 0.0808081 ... 1.91919 1.93939 1.9596 1.9798   2 

